# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the navigation task. This task has been taken from Udacity-Deep-Reinforcement-Learning Nanodegree. The algorithm you will be implementing is **REINFORCE** to achieve the task.

(https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="./Banana_Linux/Banana.x86_64") #put path to environment file

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
from agent import Agent
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from collections import deque
import numpy as np


agent = Agent(state_size, action_size, env, './model_param', 0)

n_episodes = 10000
GAMMA = 1.0

scores_deque = deque(maxlen=100)
scores = []
count = 0

for i_episode in range(1, n_episodes+1):
    env_info = env.reset(train_mode=True)[brain_name] # reset the environment
    state = env_info.vector_observations[0]            # get the current state
    
    saved_log_probs, rewards = agent.learn(state, brain_name)
    
    scores_deque.append(sum(rewards))
    scores.append(sum(rewards))

    discounted_rewards=[]
    for t in range(len(rewards)):
        Gt = 0 
        i = 0
        for r in rewards[t:]:
            Gt += GAMMA**i * r
            i += 1
        discounted_rewards.append(Gt)
    discounted_rewards = torch.tensor(discounted_rewards)    
    
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)
    
#     discounts = [gamma**i for i in range(len(rewards)+1)]
#     R = sum([a*b for a,b in zip(discounts, rewards)])

    policy_loss = []
    for log_prob, Gt in zip(saved_log_probs, discounted_rewards):
        policy_loss.append(- log_prob * Gt)
    policy_loss = torch.cat(policy_loss).sum()

    agent.optimizer.zero_grad()
    policy_loss.backward()
    agent.optimizer.step()
    
    torch.save(agent.anetwork.state_dict(), './model_param')
        
    if(i_episode%50 == 0):
        print("Episode: "+str(i_episode)+"\tAvg. Reward: "+str(np.mean(scores_deque)))
        
    if(np.mean(scores_deque) >= 13):
        print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
        break
        



Episode: 50	Avg. Reward: 0.48
Episode: 100	Avg. Reward: 0.5
Episode: 150	Avg. Reward: 0.52
Episode: 200	Avg. Reward: 0.5
Episode: 250	Avg. Reward: 0.6
Episode: 300	Avg. Reward: 0.6
Episode: 350	Avg. Reward: 0.68
Episode: 400	Avg. Reward: 0.76
Episode: 450	Avg. Reward: 0.8
Episode: 500	Avg. Reward: 0.82
Episode: 550	Avg. Reward: 0.88
Episode: 600	Avg. Reward: 1.3
Episode: 650	Avg. Reward: 0.92
Episode: 700	Avg. Reward: 1.0
Episode: 750	Avg. Reward: 1.18
Episode: 800	Avg. Reward: 1.32
Episode: 850	Avg. Reward: 1.18
Episode: 900	Avg. Reward: 1.68
Episode: 950	Avg. Reward: 1.58
Episode: 1000	Avg. Reward: 1.74
Episode: 1050	Avg. Reward: 1.6
Episode: 1100	Avg. Reward: 2.06
Episode: 1150	Avg. Reward: 1.88
Episode: 1200	Avg. Reward: 2.2
Episode: 1250	Avg. Reward: 1.96
Episode: 1300	Avg. Reward: 1.98
Episode: 1350	Avg. Reward: 2.12
Episode: 1400	Avg. Reward: 2.8
Episode: 1450	Avg. Reward: 2.78
Episode: 1500	Avg. Reward: 2.96
Episode: 1550	Avg. Reward: 3.06
Episode: 1600	Avg. Reward: 2.96
Episod

NameError: name 'score' is not defined

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```